---
title: "Weavingspace: a new way to make multivariate maps"
author: 
  - name: David O'Sullivan
    affiliations: 
      - name: Geospatial Stuff
      - name: University of Auckland
format: 
  revealjs:
    theme: [serif, custom.scss]
    transition: none
    navigation-mode: vertical
    fragment-in-url: true
    slide-number: c/t
    center: true
    logo: images/logo.png
    footer: "FOSS4G Oceania 2025"
    self-contained: true
jupyter: python3
execute: 
  freeze: auto
  cache: true
---

```{r}
#| echo: false
#| output: false
library(reticulate)
use_condaenv("weavingspace")
```

In [ ]:
#| echo: false
import os
os.chdir("/Users/david/Documents/code/weaving-space")

import matplotlib.pyplot as pyplot
import geopandas as gpd

from shapely.geometry import Polygon
from shapely.affinity import rotate

from weavingspace import TileUnit
from weavingspace import TileShape  
from weavingspace import WeaveUnit
from weavingspace import Tiling
from weavingspace import tiling_utils

ak = gpd.read_file("examples/data/imd-auckland-sa2-2018.gpkg")
ids = list("abcd")
vars = ["employment", "income", "crime", "housing"]
cmaps = 4 * ["inferno_r"]

# Mapping multivariate data

A hard problem

Many methods have been tried

We've proposing another based on tiling

## {background-color="white"}
::: {.r-stack}
![](images/small-multiples.png "Tufte recommends this approach"){.fragment .fade-out fragment-index=1 width="800"}

![](images/bivariate-choropleth-stevens-qgis.png "2-colours, OK"){.fragment .fade-in-then-out fragment-index=1 width="800"}

![](images/tricolore.png "3-colours, brown"){.fragment .fade-in-then-out fragment-index=2 width="800"}

![](images/choropleth-with-symbols-qgis.png "Probably the most conventional option"){.fragment .fade-in-then-out fragment-index=3 width="800"}

![](images/dorling-chernoff-faces.jpg "Probably the least conventional option"){.fragment .fade-in-then-out fragment-index=4 width="500"}

![](images/dot-map.png "This approach is older than you might imagine, see Jenks GF, 1953, “‘Pointilism’ as a cartographic technique” The Professional Geographer 5(5) 4–6
"){.fragment .fade-in fragment-index=5 width="800"}
:::

# Enter `weavingspace` {.smaller background-image="images/detailed-weave.png" background-opacity=0.5 style="text-shadow: 0px 0px 5px #fff;"}

## TL;DR;

Two types of thematic map for complex multivariate data

Elements in a tiled pattern 'pick up' data from area units to allow combining multiple choropleths in a single map

&nbsp;   

::: aside
Maps on the next slide are of the IMD. See: Exeter DJ, Zhao J, Crengle S, Lee A, Browne M, 2017, [The New Zealand Indices of Multiple Deprivation (IMD): A new suite of indicators for social and health research in Aotearoa, New Zealand](https://dx.plos.org/10.1371/journal.pone.0181260) _PLOS ONE_ **12**(8) e0181260
:::


## {transition="none" background-color="white"}


In [ ]:
tile_unit = TileUnit(tiling_type = "laves", code = "4.8.8",
                     crs = ak.crs, spacing = 500)

tiling = Tiling(tile_unit, ak)

fig = tiling.get_tiled_map(prioritise_tiles = True) \
  .render(ids_to_map = ids, vars_to_map = vars, colors_to_use = cmaps,
          legend = False, figsize = (10, 7))

## {transition="none" background-color="white"}


In [ ]:
tile_unit = WeaveUnit(weave_type = "twill", n = 3, strands = "ab|cd",
                      aspect = .75, crs = ak.crs, spacing = 150)
# tile_unit = tile_unit.inset_elements(10)
tile_unit = tile_unit.transform_rotate(30)
tiling = Tiling(tile_unit, ak)

fig = tiling.get_tiled_map(prioritise_tiles = True) \
  .render(ids_to_map = ids, vars_to_map = vars, colors_to_use = cmaps,
          legend = False, figsize = (10, 7))

# How it works
`weavingspace` is a python module developed for making tiled maps

[github.com/DOSull/weaving-space](https://github.com/DOSull/weaving-space)

Uses `geopandas` and a _lot_ of geometry, leveraging `shapely`

Implements many options (well over a hundred tilings and weave tiling patterns)

##


In [ ]:
#| echo: true

import geopandas as gpd
from weavingspace import TileUnit   # <1>  
from weavingspace import Tiling     # <2>

tile = TileUnit(tiling_type = "cairo", spacing = 500, 
                crs = ak.crs).inset_prototile(10)
tile.plot(r = 1, show_vectors = True)

1. A `TileUnit` stores geometry of a tiling and translation vectors so we can tile a map.
2. A `Tiling` is the result of applying a `TileUnit` to a geospatial dataset.

## The tiling process
<!-- The `Tiling` holds a `TileGrid` which determines where to translate copies of the tileable units to form a tiled map.

![](images/the-tiling-process.png)-->

::: {.r-stack}

![](images/the-tiling-process-1.png){width="1482"}

![](images/the-tiling-process-2.png){.fragment .fade-in fragment-index=1 width="1482"}

![](images/the-tiling-process-3.png){.fragment .fade-in fragment-index=2 width="1482"}

![](images/the-tiling-process-4.png){.fragment .fade-in fragment-index=3 width="1482"}

![](images/the-tiling-process-5.png){.fragment .fade-in fragment-index=4 width="1482"}

![](images/the-tiling-process-6.png){.fragment .fade-in fragment-index=5 width="1482"}
:::

##


In [ ]:
#| echo: true

tiling = Tiling(tile, ak) # <1> 
tiled_map = tiling.get_tiled_map(prioritise_tiles = True)

tiled_map.legend        = False # <2>
tiled_map.ids_to_map    = list("abcd")
tiled_map.vars_to_map   = ["income", "employment", "crime", "housing"]
tiled_map.colors_to_use = ["Reds", "Blues", "Greens", "Greys"]

fig = tiled_map.render() # <3>

1. Make the tiling and get a map.
2. Set up a bunch of parameters for the final map.
3. Render.

# Many options {.smaller background-image="images/detailed-weave.png" background-opacity=0.5 style="text-shadow: 0px 0px 5px #fff;"}

## Biaxial weaves {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(2, 3, layout = "constrained")
types = ["plain", "twill", "twill", "basket", "basket", "basket"]
strands = ["a|b", "ab|cd", "a-b|cd", "ab|cd", "abc|def", "ab-|cde"]
angles = [15, 15, 15, 15, 15, 15]
for i, (typ, strands, angle) in enumerate(zip(types, strands, angles)):
    ax = axes[i // 3, i % 3]
    u = WeaveUnit(
      weave_type = typ, strands = strands, aspect = 0.8).transform_rotate(angle)
    u.plot(ax = ax, r = 0, show_ids = False, 
            show_reg_prototile = False,
            show_prototile = False, cmap = "tab20")
    ax.set_axis_off()
    ax.set_title(f"{typ} {strands}")

## Triaxial weaves {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(1, 3, layout = "constrained")
types = ["cube", "cube", "cube"]
strands = ["-a-|-b-|-c-", "a-b|c-d|e-f", "abc|def|ghi"]
angles = [15, 15, 15, 15, 15, 15]
for i, (typ, strands, angle) in enumerate(zip(types, strands, angles)):
    ax = axes[i]
    u = WeaveUnit(
      weave_type = typ, strands = strands, aspect = 0.8).transform_rotate(angle)
    u.plot(ax = ax, r = 0, show_ids = False, 
            show_reg_prototile = False,
            show_prototile = False, cmap = "tab20")
    ax.set_axis_off()
    ax.set_title(f"{typ} {strands}")

## Archimedean tilings {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(2, 4, layout = "constrained")
codes = ["3.3.3.3.6", "3.3.3.4.4", "3.3.4.3.4", "3.4.6.4", "3.6.3.6", "3.12.12", "4.6.12", "4.8.8"]
for i, code in enumerate(codes):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = "archimedean", code = code)
    u.plot(ax = ax, r = 1, show_ids = False, 
           show_reg_prototile = True,
           show_prototile = False)
    ax.set_title(f"[{u.code}]")
    ax.set_axis_off()

## Laves tilings {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(2, 4, layout = "constrained")
codes = ["3.3.3.3.6", "3.3.3.4.4", "3.3.4.3.4", "3.4.6.4", "3.6.3.6", "3.12.12", "4.6.12", "4.8.8"]
for i, code in enumerate(codes):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = "laves", code = code)
    u.plot(ax = ax, r = 1, show_ids = False, 
           show_reg_prototile = True,
           show_prototile = False)
    ax.set_title(f"({u.code})")
    ax.set_axis_off()

## Square colourings {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(2, 4, layout = "constrained")
for i, n in enumerate([3, 4, 5, 6, 7, 9, 16, 25]):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = "square-col", n = n)
    u.plot(ax = ax, r = 1, show_ids = False, 
           show_reg_prototile = True,
           show_prototile = False)
    ax.set_title(f"{n=}")
    ax.set_axis_off()

## Hexagon colourings {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(2, 4, layout = "constrained")
for i, n in enumerate([3, 4, 7, 9, 10, 16, 19, 37]):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = "hex-col", n = n)
    u.plot(ax = ax, r = 1, show_ids = False, 
           show_reg_prototile = True,
           show_prototile = False)
    ax.set_title(f"{n=}")
    ax.set_axis_off()

## Square slices {background-color="white"}


In [ ]:
ns      = [2, 2,   4, 4,   5, 6, 8, 8]
offsets = [0, 0.5, 0, 0.5, 0, 0, 0, 1]
fig, axes = pyplot.subplots(2, 4, layout = "constrained")

for i, (n, offset) in enumerate(zip(ns, offsets)):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = "square-slice", n = n, offset = offset)
    u.plot(ax = ax, r = 1, show_ids = False, 
           show_reg_prototile = True,
           show_prototile = False)
    ax.set_title(f"n={u.n} offset={u.offset}")
    ax.set_axis_off()

## Hexagon slices {background-color="white"}


In [ ]:
ns      = [2, 3, 3, 6, 6, 9, 12, 12]
offsets = [0, 0, 1, 0, 1, 0,  0,  1]
fig, axes = pyplot.subplots(2, 4, layout = "constrained")

for i, (n, offset) in enumerate(zip(ns, offsets)):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = "hex-slice", n = n, offset = offset)
    u.plot(ax = ax, r = 1, show_ids = False, 
           show_reg_prototile = True,
           show_prototile = False)
    ax.set_title(f"n={u.n} offset={u.offset}")
    ax.set_axis_off()

## Square and hexagon dissections {background-color="white"}


In [ ]:
ttype = ["square-dissect"] * 4 + ["hex-dissection"] * 4
ns            = [5,  5, 9,  9, 7, 9,  9,  9]
offsets       = [0,  0, 0,  1, 0, 0,  1,  0]
offset_angles = [0, 45, 0, 45, 0, 0, 30, 30]
fig, axes = pyplot.subplots(2, 4, layout = "constrained")

for i, (tt, n, offset, offset_angle) in enumerate(zip(ttype, ns, offsets, offset_angles)):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = tt, n = n, offset = offset, offset_angle = offset_angle)
    u.plot(ax = ax, r = 1, show_ids = False, 
           show_reg_prototile = True,
           show_prototile = False)
    ax.set_title(f"{n=} {offset=} angle={offset_angle}")
    ax.set_axis_off()

## Crosses {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(2, 3, layout = "constrained")
for i, n in enumerate(range(2, 7)):
    ax = axes[i // 3, i % 3]
    u = TileUnit(tiling_type = "cross", n = n)
    u.plot(ax = ax, r = 1, show_ids = False, 
           show_reg_prototile = True,
           show_prototile = False)
    ax.set_title(f"{n=}")
    ax.set_axis_off()
axes[1, 2].set_axis_off()

## Stripes {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(2, 3, layout = "constrained")
for i, n in enumerate((3, 6, 9, 12, 15, 18)):
    ax = axes[i // 3, i % 3]
    u = TileUnit(tiling_type = "stripes", n = n).transform_scale(2**0.5, 0.5**0.5).transform_scale(.95, .9, True)
    u.plot(ax = ax, r = 1, show_ids = False, 
            show_reg_prototile = True,
            show_prototile = False)
    ax.set_title(f"n={u.n}")
    ax.set_axis_off()

## Stars and polygons {background-color="white"}


In [ ]:
types = ["star1"] * 3 + ["star2"] * 6
point_angles = [30] * 3 + [None] * 6
codes = ["33", "44", "63", "45", "464", "64", "663", "844"]

fig, axes = pyplot.subplots(2, 4, layout = "constrained")

for i, (tt, a, c) in enumerate(zip(types, point_angles, codes)):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = tt, point_angle = a, code = c)
    u.plot(ax = ax, r = 1, show_ids = False, 
            show_reg_prototile = True,
            show_prototile = False)
    ax.set_axis_off()
    ax.set_title(f"code={u.code}")

## Larger _n_ tilings {background-color="white"}


In [ ]:
fig, axes = pyplot.subplots(2, 4, layout = "constrained")
for i, code in enumerate(list("ACDEFGIJ")):
    ax = axes[i // 4, i % 4]
    u = TileUnit(tiling_type = "chavey", code = code)
    u.plot(ax = ax, r = 1, show_ids = False, 
            show_reg_prototile = True,
            show_prototile = False, cmap = "tab20")
    ax.set_axis_off()
    ax.set_title(f"{code=}")

## Modifying tiles (experimental) {background-color="white"}


In [ ]:
#| output: false
from weavingspace import Topology
import copy

tile1a = TileUnit(tiling_type = "square-colouring", n = 4)
topo1a = Topology(tile1a, True)
topo1b = topo1a.transform_geometry(True, True, "a", "zigzag_edge",
                                   n = 2, h = 0.55, smoothness = 5)
tile1b = topo1b.tileable

tile2a = TileUnit(tiling_type = "cairo").transform_rotate(45)
topo2a = Topology(tile2a, True)
topo2b = topo2a.transform_geometry(True, True, "b", 
                                   "scale_edge", sf = 4)
tile2b = topo2b.tileable

tile3a = TileUnit(tiling_type = "chavey", code = "a")
tile3a_dual = tiling_utils.get_dual_tile_unit(tile3a)
tile3b = copy.deepcopy(tile3a)
tile3b.tiles = tile3a_dual

tile4a = TileUnit(tiling_type = "square-dissection", n = 9, 
                  offset = 0, offset_angle = 0)
topo4a = Topology(tile4a, True)
tiles = list(topo4a.dual_tiles.values())
tile4b = copy.deepcopy(tile4a)
tile4b.tiles = gpd.GeoDataFrame(
  data = {"tile_id": list("abcdefghijklmnopqrstuvwxyz")[:len(tiles)]},
  geometry = gpd.GeoSeries(tiles))

In [ ]:
#| fig-align: center
fig, axes = pyplot.subplots(2, 4, layout = "constrained")
tiles = [tile1a, tile1b, tile2a, tile2b,
         tile3a, tile3b, tile4a, tile4b]
tiles2 = [tile1b, tile1a, tile2b, tile2a,
          tile3b, tile3a, tile4b, tile4a]
cmaps = ["Dark2"] * 4 + ["tab20c"] * 4
ecs =   ["w"] * 4 + ["#333333"] * 4
for (i, t), t2, cm, ec in zip(enumerate(tiles), tiles2, cmaps, ecs):
  ax = axes[i // 4, i % 4]
  t.plot(ax = ax, r = 1, show_prototile = False, 
          show_reg_prototile = False, r_alpha = 1, 
          cmap = cm, show_ids = False)
  t2.tiles.geometry.plot(ax = ax, fc = "#00000000", ec = ec, lw = 0.5)
  ax.set_xlim(-900, 900)
  ax.set_ylim(-900, 900)
  ax.set_axis_off()

# A web app {.smaller data-background="images/mapweaver-app.png" background-opacity=0.75 style="text-shadow: 0px 0px 5px #fff;"}
Make tiled and woven maps without code!

## {data-background="images/Wile-E-Coyote-1.jpg.webp"}

### Live demo time!

## {style="text-align:center;"}
### Questions? {style="font-size:144px;"} 
[github.com/DOSull/weaving-space](https://github.com/DOSull/weaving-space)

[dosull.github.io/mapweaver/app/](https://dosull.github.io/mapweaver/app/)

[dosull.github.io](https://dosull.github.io)